# Build RAG + LLM Model

In [ ]:
import minsearch as minsearch
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

In [2]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [3]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

<h3> 1- create our search engine consedering questions, text and section as information and the course as a key field</h3>

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [6]:
index.fit(documents)

In [7]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5} # Poids d'importance de chaque text field dans le moteur de recherche
    results = index.search(
            query=query,
            filter_dict={'course': 'data-engineering-zoomcamp'},
            boost_dict=boost,
            num_results=5
        )
    return results

<h3>2- After creat awer KB and our search engine, now is time to pass that to a LLM Model</h3>

<h5>2.1- Create prompt (imput od a LLM model) function</h5>

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    QUESTION: {question}
    CONTEXT:  {context} 
    """.strip()
    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

<h5>2.2 Create the LLM function</h5>

In [9]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

<h3>3- Time to play the game :)</h3>

In [10]:
def ragllm(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [11]:
query = 'how do I run kafka?'
answer = ragllm(query)
print(answer)

To run Kafka producer, consumer, kstreams, etc. in the terminal from a Java project, you can use the following command:

```bash
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java
```

Make sure to replace `<jar_name>` with the actual name of your JAR file in the project directory.
